# Fractals and Iteration

We've seen how fractals can be drawn using Turtle to draw lines one at a time.  But many fractals are whole images which would be very hard to draw using Turtle.  We'll see that one very powerful method that produces many distinctive (and famous!) fractals is *iteration* - repeatedly applying a specified function again and again.

This will place us into the purview of the discipline of *dynamical systems*.

One of the most important concepts for us is the idea of a *non-escaping set*.  This is the collection of points for which, when I repeatedly apply my function to it, it does not 'explode away'.  Let's look at the example below to show what this means, using the function $f(x) = 5x(1-x)$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def f(x):
    return 5 * x * (1 - x)

# Here we define the values so we can graph the function
xvalues = np.arange(0, 1,.001)
yvalues = f(xvalues)
plt.plot(xvalues,yvalues)

plt.axhline(y=0, color = 'g')
plt.axhline(y=1, color = 'g')
plt.axvline(x=0, color = 'g')
plt.axvline(x=1, color = 'g')
plt.show()

Note the green box we've chosen to draw.  Let's look at what happens if we apply $f$ to a starting point several times.

In [ ]:
xvalues = np.arange(0, 1,.001)
f1values = f(xvalues)
# Applying f again
f2values = f(f1values)
# And again
f3values = f(f2values)
# And again
f4values = f(f3values)

plt.plot(xvalues, f1values, color = 'b')
plt.plot(xvalues, f2values, color = 'r')
plt.plot(xvalues, f3values, color = 'y')
plt.plot(xvalues, f4values, color = 'c')


plt.axhline(y=0, color = 'g')
plt.axhline(y=1, color = 'g')
plt.axvline(x=0, color = 'g')
plt.axvline(x=1, color = 'g')

#Let's also add a limit to the range of the plot (the y-values it's showing)
plt.ylim(-3,3)

#And let's add a legend
plt.legend(["f1","f2","f3","f4"])
plt.show()

Ok, there's a lot here.  But let's look at it slowly.  Notice that in the part of the graph where $f(x)$ left the green box, $f(f(x)) = f^{\circ 2}(x)$ is even further out, and $f(f(f(x))) = f^{\circ 3}(x)$ is much further out, and so on.

What we can say is that once the graph leaves that green box, it'll never come back, so we say it *escapes*.  

Now, we're going to do a little bit of Python magic to draw the initial points that don't escape after a few applications of $f$.  The way we'll do that is by drawing the graph of $f^{\circ k}(x)$ (the $k$-th *iterate* of $x$ by $f$) and then, for each point $x$ we'll also plot it if $0 \leq f^{\circ k}(x) \leq 1$ (this is the condition of being in the green box after $k$ steps).

In [ ]:
xvalues = np.arange(0, 1,.001)
k = 1

# We're using a for loop to repeatedly apply f
yvalues = xvalues
for _ in range(k):
    yvalues = f(yvalues) 

plt.plot(xvalues, yvalues, color = 'b')

# We restrict the range of the plot so that it only lies within the green box now
# So the graph leaves the box exactly when it goes outside the plot window
plt.xlim(0,1)
plt.ylim(0,1)

# This list comprehension gives us a list consisting of elements from xvalues,
# included only if the graph above that point lies between 0 and 1.
non_escaping = [xvalues[i] for i in range(len(xvalues)) if yvalues[i] >= 0 and yvalues[i] <= 1]
non_escaping = np.array(non_escaping)

# We set a silly arbitrary y-value to put it for these points when they are plotted,
# just for visualization purposes, this number is just aesthetic.
val = .5
height = np.empty(len(non_escaping))
height.fill(val)

# Now, I plot the non-escaping points using a scatter plot in the same figure
plt.scatter(non_escaping, height, s = .1, color = 'r')

plt.show()

The pattern that these red non-escaping points make as $k$ is increased might look familiar - they're forming a Cantor set of the same kind that we drew earlier!

This motivates our main plan to draw fractals using iteration - we can figure out the *non-escaping set* and this will often give us a fractal!

### Drawing Julia Sets

If our goal is to find functions to iterate and find the non-escaping set for, then our next question is: *Which functions should we pick?*  One common choice are **polynomials of a complex variable**.  What does this mean?

#### Complex Numbers
Complex numbers are numbers of the form $a + i b$ where $i = \sqrt{-1}$ and $a, b$ are real numbers.  We call $a$ the *real part* and $b$ the *imaginary part* of the complex number.

Just like we can add, subtract, multiply, and divide real numbers, we can also do this to complex numbers!  This means that polynomials like $$f(x) = 5x(1-x)$$ or $$g(x) = x^4 - 17 x^2 + 2x + 13$$ can also be thought of as functions which take in complex numbers!

Often, to distinguish the fact that we want to refer to complex numbers specifically, we use the letter $z$ instead of $x$.

Much as we plot real numbers on a *number line*, since there are two real numbers in every complex number ($a$ and $b$) we plot it in the $xy$-plane, where $x$ describes the real part and $y$ the imaginary part.

Most significant for us is that, much like real numbers have an absolute value $|x|$, so too complex numbers have a *modulus*, which is essentially the same thing - it tells how far the number is from 0 (how 'big' it is).  $$abs(a + ib) = |a + ib| = \sqrt{a^2 + b^2}$$

#### Filled Julia Sets

When the thing we're iterating is a complex polynomial, we call the non-escaping set the *Filled-Julia Set* (the boundary is the *Julia set*).  Now, every problem might not have a green box like our example $f(x)$ we looked at above.  But, it stands to reason that if our complex number after a few iterations has very big modulus: above 10 or maybe 100, for example, then it is probably going to escape to $\infty$.  

So, here's the strategy:
1. Divide a rectangular region of the complex $xy$-plane into pixels.
2. Build a 2d-array corresponding to the 2d-grid of pixels in the image.  Each entry of the array is the complex number at the center of the pixel.
3. Go through the 2d-array one element at a time, iterating the complex point many times until it either escapes or we've iterated it enough times to give up. The points that do not escape after that many iterates are labeled *non-escaping*.
4. For each point, count the number of iterations it took to escape, but giving the value 0 to points that dot escape.  Build a 2d-array of these values for each pixel.
5. Color each pixel according to the number of iterates it took to escape.

We need to define a function we want to iterate.  Let's call it $g(z)$. 

We'll also define a couple of necessary parameters for us to implement the above strategy.  The below code block isn't important to the way the, but it is included here so that you can see clearly what the parameters which will be involved in this process are.

In [ ]:
# This is the function we will iterate.
def g(z):
    return z ** 2 + complex(.1, -.3) 
# complex is much like int or float, but this data type represents a complex number

# How big we're willing to let the size of a complex number get before we decide it has escaped
zabs_max = 10
# How many times we will iterate before we declare that a point will not escape
num_iterates = 80

# The minimum and maximum real part of points on the plot
xmin = -1.5
xmax = 1.5

# The minimum and maximum imaginary part of points on the plot
ymin = -1.5
ymax = 1.5

In [ ]:
# Program modified from from JeanFrancoisPuget, availible:
# https://www.ibm.com/developerworks/community/blogs/jfp/entry/My_Christmas_Gift?lang=en

import numpy as np 
from matplotlib import pyplot as plt 
from matplotlib import colors # This lets us use fancy color schemes for our pictures!

# This function takes in a complex number c and iterates for
# n from 0 to maxiter to see if z_n gets above the threshold. If it does not,
# 0 is returned, and the point is in the filled Julia set. If it 
# does, the n where it escapes is stored to add 
# color later on. See description above for more details.
def Julia(g, init_point, maxiter, zmax_abs):
    z = init_point
    for n in range(maxiter):
        if abs(z) > zmax_abs:
            return n
        z = g(z)
    return 0

# This function loops over points in the xy-plane, checking 
# whether c=x+iy is in the filled Julia set or not using the 
# function above
def Julia_Set(g, xmin,xmax,ymin,ymax,width,height,maxiter, zmax_abs):
    r1 = np.linspace(xmin, xmax, width) # x points
    r2 = np.linspace(ymin, ymax, height) # y points
    n3 = np.empty((width,height)) # in set, or n where z > 2 
    for i in range(width):
        for j in range(height):
            # Python has a built in capability for complex numbers
            # The imaginary unit is written as "1j"
            n3[i,j] = Julia(g, r1[j] + 1j * r2[i], maxiter, zmax_abs)
    return (r1,r2,n3)
 
# This function creates an image from the xy data of whether the
# point is in the Madelbrot set, or the n at which it becomes 
# larger than 2
def Julia_Image(g, xmin,xmax,ymin,ymax, zmax_abs = 10,width=10,height=10,maxiter=80,cmap='jet'):
    dpi_val = 72
    img_width = dpi_val * width
    img_height = dpi_val * height
    x,y,z = Julia_Set(g, xmin,xmax,ymin,ymax,img_width,img_height,maxiter, zmax_abs)
    
    fig, ax = plt.subplots(figsize=(width, height), dpi = dpi_val) 
    ax.imshow(z.T,cmap=cmap,origin='lower', extent = [xmin, xmax, ymin, ymax])

#-------

def g(z):
    c = .3 - 1j * .1
    return z ** 2 + c

# Set a cool looking color map, and give a size for the figure!
# Note that we can put a function in as input to another function!
Julia_Image(g, -1.5,1.5,-1.5,1.5,cmap='hot')    


Try playing with the previous cell a bit.  
- What happens if you alter the function $g(z)$?  (An easy way is just to change the complex number inside of it, but you can try more radical polynomials if you'd like)
- Or lower `num_iterates` until it is very small?  
- Or alter `z_abs` until it is either very small or very big?
- Or change the parameters of the window size?

Try a couple of values for $g(z) = z^2 + c$
- $c = -0.8 + .156i$
- $c = -.7269 + .1889i$
- $c = −0.70176− 0.3842i$

You can get almost any pattern you'd like by varying the function $g$!

In particular, try comparing the results if you use $g(z) = z^2 + c$ for $c = .1 - .3i$ vs $c = .5 - .3i$.

### The Mandelbrot Set

Those two cases have a very different heuristic appearance; if $c = .1 - .3i$ the filled Julia set is a large filled blob with only the boundary looking complicated and fractal-like.  It has a big interior, none of which escapes.  On the other hand, if $c = .5 - .3i$, then we see a fractal pattern with lots of gaps between elements - a lot like the Cantor set.

By analogy with that fractal, this is often called a *Cantor Julia set*.

The distinction between values of $c$ which have Cantor Julia set and do not turns out to have a relatively simple description.  If the Julia set of $g(z) = z^2 + c$ has a big blobby interior, then 0 does not escape to infinity, and vice versa.

Thus, for quadratics, we only need to test 0 if we want to know whether or Julia set will be big and blobby, or wispy and disconnected.

The set of $c$ for which the filled Julia set is a blob, or equivalently for which 0 does not escape, is called *The Mandelbrot Set*.  This is the same Mandelbrot that coined the term fractal that we heard of at the very beginning of today. The mathematics behind the Madelbrot set is a little tricky, and I won't explain it in detail. If you are interested, I reccomend checking out one of the great videos online explaining it, like [this Numberphile video](https://www.youtube.com/watch?v=NGMRB4O922I&feature=youtu.be).

Our goal now will be to plot the Mandelbrot set!

To do this, we'll define a sequence of points starting with $z_0 = 0$ and applying $g(z) = z^2 + c$ to each to obtain the next element of the sequence, so $$z_1 = g(z_0) = c$$ and $$z_2 = g(z_1) = c^2 + c$$ and so on with $$z_n = g(z_{n-1}) = z_{n-1}^2 + c$$.  

**Note: A key observation is that if $abs(z_n)>2$ ever happens, then the sequence will escape.  So, we can set the equivalent of `zabs_max = 2`**

As a non-coded example, consider $c=1.0+0.0i$, which is just a real number, since the imaginary part is zero, but is a perfectly good example. Then:
\begin{align}
z_0 &= 0 \\
z_1 &= 1.0 \\
z_2 &= 1^2+1=2 \\
z_3&= 2^2+1=5 \\
z_3&= 5^2+1=26, \\
\end{align}
and so on. We see that $z$ quickly diveges beyond 2, so the complex number $c=1.0+0.0i$, and therefore the point (1.0,0.0) on the $xy$-plane, is not in the Mandelbrot set. Lets try another simple example: $c=-0.5+0.0i$. Then:
\begin{align}
z_0 &= 0 \\
z_1 &= -0.5 \\
z_2 &= (-0.5)^2-0.5=-0.25 \\
z_3&= (-0.25)^2-0.5=-0.4375 \\
z_3&= (-0.4375)^2-0.5=-0.3085938, \\
\end{align}
we could keep going, but $z$ does not appear to be diverging. If we kept going, $z$ would approach a number around $-0.366$, which is less than 2, so $c=-0.5+0.0i$, or the point $(-0.5,0)$ is in the Mandelbrot set.

Ok, more than enough math, lets make a pretty picture! The program to make a Mandelbrot set is below.

In [ ]:
# Program modified from from JeanFrancoisPuget, availible:
# https://www.ibm.com/developerworks/community/blogs/jfp/entry/My_Christmas_Gift?lang=en

import numpy as np 
from matplotlib import pyplot as plt 
from matplotlib import colors 

# This function takes in a complex number c and iterates for
# n from 0 to maxiter to see if z_n > 2. If it does not,
# 0 is returned, and the point is in the Mandelbrot set. If it 
# does, the n where it becomes greater than 2 is stored to add 
# color later on. See description above for more details.
def Mandelbrot(c,maxiter):
    z = c
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return 0

# This function loops over points in the xy-plane, checking 
# whether c=x+iy is in the Madelbrot set or not using the 
# function above
def Mandelbrot_Set(xmin,xmax,ymin,ymax,width,height,maxiter):
    r1 = np.linspace(xmin, xmax, width) # x points
    r2 = np.linspace(ymin, ymax, height) # y points
    n3 = np.empty((width,height)) # in set, or n where z > 2 
    for i in range(width):
        for j in range(height):
            # Python has a built in capability for complex numbers
            # The imaginary unit is written as "1j"
            n3[i,j] = Mandelbrot(r1[i] + 1j*r2[j],maxiter)
    return (r1,r2,n3)
 
# This function creates an image from the xy data of whether the
# point is in the Madelbrot set, or the n at which it becomes 
# larger than 2
def Mandelbrot_Image(xmin,xmax,ymin,ymax,width=10,height=10,maxiter=80,cmap='jet'):
    dpi_val = 72
    img_width = dpi_val * width
    img_height = dpi_val * height
    x,y,z = Mandelbrot_Set(xmin,xmax,ymin,ymax,img_width,img_height,maxiter)
    
    fig, ax = plt.subplots(figsize=(width, height), dpi = dpi_val) 
    ax.imshow(z.T,cmap=cmap,origin='lower', extent = [xmin, xmax, ymin, ymax])

#-------


# Set a cool looking color map, and give a size for the figure!
Mandelbrot_Image(-2.0,0.5,-1.25,1.25,cmap='hot')    


## Summary and conclusions

So what was the point of all of this? Well, for one thing, we generated a lot of awesome looking patterns. Also, as was hinted at, there is complex and deep mathematics that underly these patterns. Even so, the code to generate them is quite simple; a few simple rules and maybe some randomness results in structural richness that resembles patterns that we see in nature.

The self-similarity that these fractals possess should be regarded as a fantastical property.  Being self-similar is essentially saying that these things are the same no matter what the amount of zooming in happening is.  In physics (and much of our normal lives), this isn't true - life is very different if you zoom in by a factor of $10^{10}$, as are physical laws like gravity or electromagnetism. Situations where things look the same at *all* scales have exotic interactions.  But they do happen!  One of the most common examples of this kind of scale invariance happens in materials as they undergo phase transitions!

And if you are not interested in philosophical or physical musings, at least we generated a lot of awesome-looking patterns!

## Exercises

### Julia Sets

Explore around in the Julia set and Mandelbrot a bit and find some interesting Julia sets and pieces of the Mandelbrot set!  One good way to do this is to look at the $g(z) = z^2 + c$ family, and pick points which are very close to the boundary of the Mandelbrot set.  You can also notice the satellite (very small and kind of far off from the central blob) copies of the Mandelbrot set and test values of $c$ which fall in those.

Try to mess with the matplotlib options a bit (from colormaps to image sizes and such) to gain some intuition for how it works.

### Finding Strange Attractors

The entirety of this notebook before here was dedicated to using functions where many points 'escape' to construct the fractal set consisting of the non-escaping points.  If a map has many points escaping to infinity, we might call it *repelling*.  But this is not the only possible behavior.  We can have a function, for example, for which most points don't go to infinity, but instead after being iterated a couple times approach a 'strange shape' which they move around in an unpredictable way.  This kind of 'strange shape' is called a *strange attractor*, and is of great popular, mathematical, and computational interest.  

Let's try to draw one as an exercise!

The *Henon Map* is a function of two real variables, which outputs two real variables, and has variables $a$ and $b$ as fixed parameters.  It is given by $$H(x,y) = (1 - a x^2 + y, bx).$$

First and foremost, write a function `henon(x, y, a, b)` with `a` having a default value set to $1.4$ and `b` having a default value set to $.3$.

In [ ]:
def henon(x, y, a = 1.4, b = .3):
    ''' 
        This is the Henon map, a function which takes in 2 real numbers,
        and outputs two real numbers.  a and b are real parameters
    '''
    pass

Now, we want to do something similar to what we did with the Mandelbrot set or filled Julia sets earlier.  Here, though, we're less worried about points going off to infinity.  Write a function `iterator(henon, x, y, min_iters, max_iters)` which takes as input the `henon` function you wrote above, values `x` and `y` which describe our starting point, and with `min_iters` and `max_iters` integers as follows:  This should output a $2$ by `k = max_iters - min_iters` np-array where each column consists of an $x$-value and a $y$-value representing the results of applying `henon` to $(x,y)$ many times.  The 0th element of the list should be the result after applying `henon` a number of times equal to `min_iters`, and the last element of the list should be the result of applying `henon` a number of times equal to `max_iters - 1`.  

That is to say the result of `iterator(henon, 0, 0, 2, 5)` should be 2 by 3, looking like `[henon(henon(0,0)), henon(henon(henon(0,0))), henon(henon(henon(henon(0,0))))]`.  In essence, this is computing the list of iterates of $(x,y)$ by the Henon map, but then throwing away the first several of them.

In [ ]:
import numpy as np

def iterator(henon, x, y, min_iters, max_iters):
    '''
        Computes the list of iterates of (x,y) by the Henon map, but throws away the first
        several, keeping them only after the henon map has been applied "initial" many times.
        Stops once it has been applied "final" many times.
        
        Outputs a 2 by (max_iters - min_iters) numpy array
    '''
    pass

Now that we have this, let's try to use it to find a strange attractor!  If we truly believe that most points will be 'attracted' towards this strange thing, then we should feel free to pick an initial point relatively randomly (I use the value $(x,y) = (.5, .5)$ in my code).  Then, since it might start relatively far away, but is attracted, we will throw away the first 99 iterates of the point, starting to record them only after 100 iterations, and stopping at 10000.  (If your code takes a very long time to run, there may be a more efficient way to implement your code - talk to the instructors or see if you can think of it!)

Use your iterator function written above to obtain that list, and then split it into x-values and y-values and use a scatter plot to see if you can see the strange attractor.  Set the size of scatterplot dots to small with the optional argument `s = .1` or an even smaller value.

In [ ]:
import matplotlib.pyplot as plt

# Your code here

This the Henon strange attractor!  It is another example of a fractal, with its self-similarity having to do with some complicated ways it responds to be folded and twisted up.  If you knock out even more early iterates and zoom your plot into parts of it, you'll see that it folds in on itself many many times in a very fine structure.  In fact, it's true that if you could take a pure horizontal slice of this strange attractor, you would get a (very thin) Cantor set!

### The Feigenbaum Attractor

Much as in the previous exercise, this exercise is about finding a fractal by way of attracting points.  Albeit here, there is a technical difference because we are considering a 1-parameter family of maps which take in 1 real number, rather than a function of two real variables.  However, we will brush that distinction under the rug.

Consider the quadratic function $$f(x) = a x (1-x),$$ where $a$ is our parameter and $x$ are variable.  We considered one case of this above, but now we'll restrict to $0 \leq a \leq 4$, where no repelling behavior occurs.

First, write this function in Python below as `f(x, a)`.

In [ ]:
def f(x, a):
    pass

This family of maps has weird behavior under iteration.  Try iterating $.5$ by this map a few times (run the cell below, and then the one below that several times) for a few values of $a$, starting at, for example, $a = 2$, repeating for ever larger values of $a$ until you get past $a = 3$.  What happened?

In [ ]:
point = .5
a = 3.1

In [ ]:
point = f(point, a)
print(point)

Now, just as in the Henon problem above, write an iterator function `iterator(funct, x, a, min_iters, max_iters)`.  If you have done the previous exercise, the only difference should be that because $f(x,a)$ only outputs one real number, the output of this function should be a 1-dimensional numpy array of length `max_iters - min_iters` consisting of iterates of `x` starting at `min_iters` many applications of $f$ on $x$. 

In [ ]:
import numpy as np

def iterator(funct, x, a, min_iters, max_iters):
    pass

Now, let's start with our erstwhile friend $x = .5$ and we'll endeavor to use this iterator function to make a scatter plot.  A range of a values has been provided for you below.  For each value of $a$, use iterator to produce a list starting at 100 iterations and ending at 500.  This will output a numpy array of values of `x` which are iterates of the start point.  Create a numpy array of equal length with the constant value of `a` which produced that sequence and plot the points as $(a, x)$ on a scatterplot.  Place all scatterpoints on the same plot.

Be sure to decrease the size of scatterplot dots with `s = .1` or a smaller value. 

In [ ]:
import matplotlib.pyplot as plt

x_init = .5
avalues = np.linspace(0,4,500) # If your code is slow, replace 500 with something smaller
max_iters = 500
min_iters = 100

# Your code here

This shape is called the Feigenbaum attractor!  It is fundamentally related to the way that orderly systems collapse into chaotic behavior!  It's also an example of a fractal which has *asymptotic self-similarity*, in that it only looks *very similar* to itself when zoomed in in the right places, but is increasingly close to being self-similar the further you zoom in.